# Carrying out analysis on texts on the topic of depression and the emotions felt

The XED dataset (https://github.com/Helsinki-NLP/XED) was used to train the emotion classifier. The emotion classifier was trained and sourced from an app example of "Multilingual Sentiment Analysis" on Cohere's website (https://docs.cohere.com/page/multilingual-sentiment-analysis).

A little bit about the XED dataset:
The XED dataset consists of annotated movie subtitles. The annotations consists of these emotions: anger, anticipation, disgust, fear, joy, sadness, surprise, and trust. The topics of sentences vary. Sentence length distribution is:
- 1-31 words: 41.9%
- 31-61 words: 42.2%
- 61-91 words: 11.9%

Initially, I explored a dataset of tweets. Tweets initiate conversation and it's max character length of 280 allowed a max word count of roughly between 40 - 70 words. This made it somewhat similar to the XED dataset. The dataset I used (https://www.kaggle.com/datasets/gargmanas/sentimental-analysis-for-tweets/data) was labelled with a keyword search.

Here, we are importing all the modules we need as well as loading a dataset of tweets into a dataframe.

In [3]:
import pandas as pd
import cohere
import torch
import pickle
import sklearn
import numpy
pd.set_option('display.max_colwidth', None)

# Load the reddit and stock data into a dataframe
df = pd.read_csv("data/sentiment_tweets3.csv")

## Exploring the Twitter dataset

In [4]:
# Top 5 rows of the dataset. There doesn't seem to be an order to the dataset
df.head()

,Index,message to examine,label (depression result)
0,106,"just had a real good moment. i missssssssss him so much,",0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http://www.youtube.com/watch?v=zoGfqvh2ME8,0
3,288,"@lapcat Need to send 'em to my accountant tomorrow. Oddly, I wasn't even referring to my taxes. Those are supporting evidence, though.",0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [5]:
# Nothing out of the ordinary
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10314 entries, 0 to 10313
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Index                      10314 non-null  int64 
 1   message to examine         10314 non-null  object
 2   label (depression result)  10314 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 241.9+ KB


In [8]:
# 2314 rows where text is labelled as depression
df[df["label (depression result)"]==1].value_counts()

Index   message to examine                                                                                                                                                                                                                                                                                                        label (depression result)
800000  The lack of this understanding is a small but significant part of what causes anxiety & depression to both feel so incredibly lonely. It's soooo easy to compare. It's so easy to invalidate ourselves because of that.                                                                                                   1                            1
801590  I've got a black eye of a soulMorals in a hole Wish I was still dead But the tv said it's just a tropical depression                                                                                                                                                                               

In [9]:
# A sample of tweets labelled as depression
dep_rows = df[df["label (depression result)"]==1]
dep_rows.sample(10)

,Index,message to examine,label (depression result)
9167,801167,@awg_allan @TelegramSam100 I'll agree to disagree. Solving the self-esteem and depression issue stops him from ever getting to the murderous obsession (imo).,1
8064,800064,Depression and No tears left to cry snatched me bold pic.twitter.com/Fxw7YgNmwY,1
9237,801237,"For some people, psychiatric diagnosis is helpful, and the problem is that it was not given early enough. For others, a diagnosis is deeply oppressive. What's your experience? http://bit.ly/2JmldiAÂ #Psychiatry #MentalIllness #Schizophrenia #Depression #Borderline #OCD",1
10068,802068,"It's a cliche, but I hope you'll start a nuclear war to cure your depression",1
9396,801396,"@asymmetricinfo @ThePlumLineGS It is almost the definition of anti-capitalist. No, that doesn't mean that it can't be considered as a short term sunsetting fix for a depression. So why would we discuss it now?",1
9759,801759,The weather outside today is what depression looks like.,1
8195,800195,"Who wants to see me sing! Created for Stand-Up When, I present Crash! The Musical!, a multi-media number set in the great-depression right after the stock market crash of 1929. It's depression-tastic! pic.twitter.com/nEXTt0G5VZ",1
10220,802220,Today's a day where my depression is just a huge thing over my head,1
10305,802305,RT Depression Could Be Improved With Vitamin D Deficiency Treatment <Emoji: Rightwards arrow> http://aboutdepressionfacts.com/4wxuÂ pic.twitter.com/QGgbqPZUMR #health #well,1
8465,800465,@JustGiving I've #justdonated to develop a charity to further the understanding and treatment of anxiety and depression in young people so that other lives may be saved.. Donate on @justgiving and help raise Â£10000 https://www.justgiving.com/crowdfunding/youngpeoplesmentalhealth?utm_source=twitter&utm_medium=socpledgedesktop&utm_content=youngpeoplesmentalhealth&utm_campaign=post-pledge-desktop&utm_term=qveqXZaEyÂ â¦,1


I came to the conclusion that a dataset consisting of tweets is not a great dataset to use in this scenario, given tweets tend to consist of news and other topics unrelated to conversation or thoughts. Sometimes people tweet about other people's thoughts.

An example is the second last row. It seems to be a link to an article about treatment for depression. This could also be due to the keyword search not being the best method for labelling. The source of our data is incredibly important for an issue that is talked a lot about on social media.

Instead of taking comments or tweets, I looked at Reddit post titles. Post titles are basically summaries of the post. Below, I start exploring a dataset collected from various Reddit forums focused on depression-related topics (https://www.kaggle.com/datasets/diegosilvadefrana/depression-dataset). Reddit post titles focused on depression-related topics tend to be posts looking for support, conversation or a connection with others who may be experiencing similar struggles. Hence, by looking at post titles and carrying out sentiment analysis, we can aim to understand people struggling with depression better by understanding their emotions.

## Exploring the Reddit dataset

In [4]:
df_reddit = pd.read_csv("data/dataset.csv")
df_reddit.head()

title  \
0                                                                                 Regular check-in post, with information about our rules and wikis   
1  Our most-broken and least-understood rules is "helpers may not invite private contact as a first resort", so we've made a new wiki to explain it   
2                                                                                        Going back to college at 33 after 3 times of dropping out.   
3                                                                                                                         Crying alone all the time   
4                                                     I genuinely don’t think I’ll ever be able to handle all of the adult responsibilities I have.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

I'm going to clean the data by removing the post content, as the sentence length distribution is far too different to the data the model was trained on. The score of the publication is also unnecessary.

In the future, I plan to use Cohere's `Summarize` to summarize the content of the post. I can then compare this data to the current dataset and to itself.

I will also remove the first 2 posts as they are introduction posts to the subreddit /r/depression.

In [5]:
df_reddit = df_reddit.drop(['content', 'score'], axis=1)
df_reddit.head()

,title
0,"Regular check-in post, with information about our rules and wikis"
1,"Our most-broken and least-understood rules is ""helpers may not invite private contact as a first resort"", so we've made a new wiki to explain it"
2,Going back to college at 33 after 3 times of dropping out.
3,Crying alone all the time
4,I genuinely don’t think I’ll ever be able to handle all of the adult responsibilities I have.


In [6]:
df_reddit = df_reddit.drop([0, 1])
df_reddit.head()

,title
2,Going back to college at 33 after 3 times of dropping out.
3,Crying alone all the time
4,I genuinely don’t think I’ll ever be able to handle all of the adult responsibilities I have.
5,I'm not depressed anymore yay
6,"Ever since I started meds, everyone acts like I’m not allowed to have negative feelings."


Exploring the dataset, briefly, further

In [85]:
df_reddit.sample(5)

,title
8224,hopeless. sorrowful. and lost.
10369,need to talk to someone..... saw someone that I have issues with.
425,I hate how life isn’t perfect
6375,STOP THE COMPETITVENESS
3242,damn I'm crying


In [86]:
df_reddit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12454 entries, 2 to 12455
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   12454 non-null  object
dtypes: object(1)
memory usage: 97.4+ KB


In [87]:
# Some post titles are bound to be repeated, with some people sharing the same feelings
# The emotions associated with frequent pieces of text can be analysed further to create relationships between certain keywords and emotions
df_reddit.describe()

,title
count,12454
unique,12099
top,I need help
freq,8


In [88]:
# Prepping to create embeddings
[list(df_reddit['title'])[0]]

['Going back to college at 33 after 3 times of dropping out.']

## Getting the embeddings

In [8]:
api_key = ""

co = cohere.Client(api_key)

In [ ]:
chain_model = pickle.load(open('models/trained_emotion_classifier.pkl', 'rb'))

# Using Cohere's powerful 'embed' method to be able to compute the text data
# NOTE: only passed in one piece of text due to a limit on API calls
text_embeddings = co.embed(texts=[list(df_reddit['title'])[0]],
                           model='multilingual-22-12',
                           truncate='RIGHT').embeddings

# Executing the model to predict probabilities of emotions with predict_proba (method in scikit-learn that returns probabilities
# using logistic regression since the trained classifier uses logistic regression)
outputs = torch.as_tensor(chain_model.predict_proba(text_embeddings),
                          dtype=torch.float32)

# Sorts probability outputs associated with the text input where the strongest emotions are first
probas, indices = torch.sort(outputs, descending=True)

# The tensors are converted from GPU-based to CPU-based as NumPy operates on CPU
probas = probas.cpu().numpy()[0]
indices = indices.cpu().numpy()[0]

In [10]:
# Probabilities associated with emotions
# (numbers associated with emotions are shown below, from 0 - 7)
print(probas)
print(indices)

[0.5975665  0.39681658 0.26276317 0.1869488  0.10719531 0.10698901
 0.09613472 0.02223289]
[4 5 7 1 0 3 2 6]


In [12]:
index_two = df_reddit.iloc[0]

# Assigning text labels to the classified emotions and appending new emotion columns to the dataframe
index_two['Anger'] = probas[numpy.where(indices == 0)[0][0]]
index_two['Anticipation'] = probas[numpy.where(indices == 1)[0][0]]
index_two['Disgust'] = probas[numpy.where(indices == 2)[0][0]]
index_two['Fear'] = probas[numpy.where(indices == 3)[0][0]]
index_two['Joy'] = probas[numpy.where(indices == 4)[0][0]]
index_two['Sadness'] = probas[numpy.where(indices == 5)[0][0]]
index_two['Surprise'] = probas[numpy.where(indices == 6)[0][0]]
index_two['Trust'] = probas[numpy.where(indices == 7)[0][0]]

index_two['Top 3 Emotions'] = ["Joy", "Sadness", "Trust"]

index_two

title             Going back to college at 33 after 3 times of dropping out.
Anger                                                               0.107195
Anticipation                                                        0.186949
Disgust                                                             0.096135
Fear                                                                0.106989
Joy                                                                 0.597566
Sadness                                                             0.396817
Surprise                                                            0.022233
Trust                                                               0.262763
Top 3 Emotions                                         [Joy, Sadness, Trust]
Name: 2, dtype: object

You don't need to carry out sentiment analysis to realise emotions felt throughout a depressive period isn't just sadness. Sentiment analysis can be both very important and useful, as well as redundant. Of-course, the topic of the inputs, the different emotion classifications in the training dataset, and source of text are examples of aspects that can change how useful sentiment analysis can be.

In this example, the largest probability is associated with "Joy". An interesting point is that "Sadness" is the emotion associated with the second largest probability. I don't think that makes either emotion redundant.

If I had the ability to be able to predict probabilities of emotions of all the datapoints, I would carry out some of the following statistical tests:
- looking at relationships between emotions
- looking at relationships between keywords and emotions

In a study carried out on Twitter, it was found that users with depression exhibited distinct tweeting patterns (https://www.psypost.org/2021/12/an-analysis-of-twitter-posts-suggests-that-people-with-depression-show-increased-rumination-on-social-media-overnight-62306). These patterns can be explored in the texts and emotions exhibited in this dataset.